In [ ]:
from tensorflow.python.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, Flatten, Dropout, Dense
from tensorflow.python.keras.models import Model
import numpy as np


import tensorflow as tf
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.callbacks import History
from lib.KerasHelpers.modelhelpers import model_placement
from src.models.model import TextCNN
from argparse import ArgumentParser
import deepdish as dd
import numpy as np
import os

    
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split



## this first thing is to get the word_embeding matrix (embeding_weights)

In [ ]:
# this funtion is not used here, but recommend to use in real cases
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

def get_word_vectors(word_index, word2vec_flag=False):

    counter = 0

    num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

    if not word2vec_flag:

        embeddings_index = {}
        f = open(os.path.join('data/raw/wordvectors/glove.6B', 'glove.6B.300d.txt'))
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

        for word, i in word_index.items():
            if i >= MAX_NUM_WORDS:
                continue

            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
                counter = counter + 1
    else:

        model = KeyedVectors.load_word2vec_format('data/raw/wordvectors/GoogleNews-vectors-negative300.bin', binary=True)

        num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
        for word, i in word_index.items():
            if i>=MAX_NUM_WORDS:
                continue
            try:
                embedding_vector = model[word]
                embedding_matrix[i] = embedding_vector
                counter = counter + 1
            except KeyError:
                embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), EMBEDDING_DIM)
                
    print ('Found %s word vectors.' % counter)
    return embedding_matrix

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

print('Preparing embedding matrix.')
embedding_matrix = get_word_vectors(word_index, word2vec_flag=True)
# word2vec true for the google word2vec, false for the glove

print('Shape of embedding matrix:', embedding_matrix.shape)

# fname_wordvec = 'glove_'
fname_wordvec = 'word2vec_'

np.save(os.path.join(processed_DIR, fname_wordvec+'embedding.npy'), embedding_matrix, allow_pickle=False)

print ('Saved embedding.npy and data.h5 in %s' % processed_DIR)

## model

In [ ]:
# chainwise implementation

def TextCNN(MAX_SEQUENCE_LENGTH, num_classes, num_words, dropout_rate=0.4, flag='rand', embedding_weights=None):

    """
    - flag: 'rand':embedding layer train from scrath
            'non': finetune the predefined embedding layer
            others: use predefined embedding layer
    """
    EMBEDDING_DIM = 300
        
    x_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

    if 'rand' in flag:
        embedding_layer = Embedding(input_dim=num_words,
                                    output_dim=EMBEDDING_DIM,
                                    embeddings_initializer='uniform',
                                    input_length=MAX_SEQUENCE_LENGTH)
        
    else:
        trainable_flag = False
        
        if 'non' in flag: trainable_flag = True
        
        embedding_layer = Embedding(input_dim=num_words,
                                    output_dim=EMBEDDING_DIM,
                                    weights=[embedding_weights],
                                    input_length=MAX_SEQUENCE_LENGTH,
                                    trainable=trainable_flag)

    x = embedding_layer(x_input)

    kernel_sizes = [3, 4, 5]
    pooled = []

    for kernel in kernel_sizes:

        conv = Conv1D(filters=100,
                      kernel_size=kernel,
                      padding='valid',
                      strides=1,
                      kernel_initializer='he_uniform',
                      activation='relu')(x)
        
        pool = MaxPooling1D(pool_size=MAX_SEQUENCE_LENGTH - kernel + 1)(conv)

        pooled.append(pool)

    merged = Concatenate(axis=-1)(pooled)

    flatten = Flatten()(merged)

    drop = Dropout(rate=dropout_rate)(flatten)
    
    x_output = Dense(num_classes, kernel_initializer='he_uniform', activation='softmax')(drop)

    return Model(inputs=x_input, outputs=x_output)

In [ ]:
dataset_DIR = 
drop_rate = 0.2
flag      = 'rand'
wordvectores = False
batch_size   = 32
num_epochs   = 12
lr = 0.001

kwargs = {'MAX_SEQUENCE_LENGTH': x_train.shape[1],
          'num_classes': y_train.shape[1],
          'num_words': data_dict['num_words'],
          'dropout_rate': dropout_rate,
          'flag': flag
         }

if 'rand' in mode:
    kwargs['embedding_weights'] = None

else:
    fname_wordvec = 'glove_'

    if wordvectors:
        fname_wordvec = 'word2vec_'

    kwargs['embedding_weights'] = np.load(os.path.join(dataset_DIR, fname_wordvec+'embedding.npy'))

text_model = TextCNN(**kwargs)

# model = model_placement(text_model, num_gpus=options.num_gpus)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=SGD(lr=lr))

model.fit(x=x_train, y=y_train, 
          batch_size=batch_size, 
          epochs=num_epochs, 
          validation_data=validation_data)